# 知识图谱数据调用探索

- 参考:
    - https://kq4b3vgg5b.feishu.cn/wiki/MCs6wDHyMiAJp5kHVv7cKsnqnLf?table=tbl8Hhg2lpLG6EM0&view=vewDvZhc5C
    - https://blog.csdn.net/2401_85378759/article/details/144713235
    - https://blog.csdn.net/Andy_shenzl/article/details/145590829

In [1]:
import os
import pandas as pd

In [2]:
import tiktoken

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)

from graphrag.query.question_gen.local_gen import LocalQuestionGen
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore

D:\Miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
INPUT_DIR = "../graphRag/output"
LANCEDB_URI = f"{INPUT_DIR}/lancedb"

COMMUNITY_REPORT_TABLE = "community_reports"
COMMUNITY_TABLE = "communities"
ENTITY_TABLE = "entities"
DOCUMENT_TABLE = "documents"
RELATIONSHIP_TABLE = "relationships"
TEXT_UNIT_TABLE = "text_units"
COMMUNITY_LEVEL = 2

## 实体

In [4]:
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
print(f"Entity count: {len(entity_df)}")
entity_df.head(3)

Entity count: 1021


,id,human_readable_id,title,type,description,text_unit_ids,frequency,degree,x,y
0,7d8dfeeb-d5f6-4f25-ae86-f69c7994249e,0,绍兴,GEO,绍兴是中国浙江省的一个历史名城，宋代时期就已经非常重要。李清照，作为宋代著名的女词人，她的生...,[ed0d7a7cec224017e5cfe7df292b6d6a031967471fba1...,4,7,0.0,0.0
1,73125e53-678d-4ca9-b526-7679bf230aec,1,高宗,PERSON,宋高宗赵构，南宋初期的皇帝，同时也是宋真宗的第九子。在金军入侵北宋后期和南宋初期，国家动荡不...,[ed0d7a7cec224017e5cfe7df292b6d6a031967471fba1...,9,9,0.0,0.0
2,615e99dc-26e7-4a59-b424-b628ea596305,2,1129年,DATE,1129年，即建炎三年，李清照46岁。这一年对她来说意义重大，不仅因为她在这一年创作了晚期的...,[ed0d7a7cec224017e5cfe7df292b6d6a031967471fba1...,5,8,0.0,0.0


In [33]:
print(entity_df.head(3))

                                     id  human_readable_id  title    type  \
0  7d8dfeeb-d5f6-4f25-ae86-f69c7994249e                  0     绍兴     GEO   
1  73125e53-678d-4ca9-b526-7679bf230aec                  1     高宗  PERSON   
2  615e99dc-26e7-4a59-b424-b628ea596305                  2  1129年    DATE   

                                         description  \
0  绍兴是中国浙江省的一个历史名城，宋代时期就已经非常重要。李清照，作为宋代著名的女词人，她的生...   
1  宋高宗赵构，南宋初期的皇帝，同时也是宋真宗的第九子。在金军入侵北宋后期和南宋初期，国家动荡不...   
2  1129年，即建炎三年，李清照46岁。这一年对她来说意义重大，不仅因为她在这一年创作了晚期的...   

                                       text_unit_ids  frequency  degree    x  \
0  [ed0d7a7cec224017e5cfe7df292b6d6a031967471fba1...          4       7  0.0   
1  [ed0d7a7cec224017e5cfe7df292b6d6a031967471fba1...          9       9  0.0   
2  [ed0d7a7cec224017e5cfe7df292b6d6a031967471fba1...          5       8  0.0   

     y  
0  0.0  
1  0.0  
2  0.0  


In [5]:
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_TABLE}.parquet")
print(f"Entity count: {len(entity_embedding_df)}")
entity_embedding_df.head(3)

Entity count: 128


,id,human_readable_id,community,level,parent,children,title,entity_ids,relationship_ids,text_unit_ids,period,size
0,9eb41439-c5d3-4cde-9627-2395f12776d9,0,0,0,-1,"[40, 41, 42, 43, 44, 45, 46, 47, 48]",Community 0,"[fa0fdf70-4d48-4f9a-a0d6-156ebd898e4d, e935cc0...","[0059ae6c-34ce-49c1-b796-76f09638686d, 040d1e7...",[02b3e0c7578ca98f4ade399261fe7f80bd1515a253117...,2025-03-28,272
1,62ae8873-9aa1-4668-b8b4-546c00aa1b90,1,1,0,-1,[],Community 1,"[d339717a-607a-4b01-83a6-3f9ebb6ac851, 966b71a...","[190f6382-aadb-476d-9038-7b92658eee4b, c6ec38b...",[cc3ca7cf89d791f5245c6eace9b945ae9d04a805fec2b...,2025-03-28,3
2,b6458935-a64e-4182-b08e-599ebe900bea,2,2,0,-1,[],Community 2,"[7748185d-cf93-4df8-995d-65f10bd45da3, 72612cd...","[0d0adf0d-c0ff-4d46-8092-12caf5cb7ae1, 6aa1366...",[63006da44a5bc1180ed270ca98b03db18891c028fe6d3...,2025-03-28,3


In [6]:
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)
entities[:3]

[Entity(id='006872d9-58ba-4f89-a9ce-40e69b382ffd', short_id='994', title='梧桐树', type='EVENT', description='在诗中常用来象征秋天和离愁别绪的树木', description_embedding=None, name_embedding=None, community_ids=['34', '110'], text_unit_ids=['7f32a0860d5cbbfd6a170edfad5cde3369f292d95925658541c013279825eaab075ce1b7f944c67e18cbb6aad503e91395a44236f8407fb8de710b034f8f8106'], rank=3, attributes=None),
 Entity(id='0133c5a3-aa3e-474c-a28a-531a261937d3', short_id='805', title='中秋', type='EVENT', description='中国传统节日，桂花在此时节开放，被李清照称为花中之冠', description_embedding=None, name_embedding=None, community_ids=['34', '109'], text_unit_ids=['f44a1440a909266504e060ecd6b8f3998076e48084e3897de1131a0c88cc443de254a854a17aaae039ede00c11ff973c73d87bf21e4e1340cca17c1e28218b0e'], rank=1, attributes=None),
 Entity(id='013f2f31-bf4e-4c3b-bcb7-e3a4ba835f10', short_id='228', title='建康城', type='GEO', description='建康城，即古代的秣陵，是李清照生活和创作的重要地点之一。1128年12月，这位宋代女诗人曾登上建康城寻找诗歌的灵感。建康城不仅在李清照的生命轨迹中占据了一席之地，还与整个南宋历史背景紧密相连，见证了那个时代的文化与社会动态。李清照在那里的探访也反映了宋代知

## 关系

In [7]:
relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
relationships = read_indexer_relationships(relationship_df)
print(f"Relationship count: {len(relationship_df)}")
relationship_df.head(3)

Relationship count: 1138


,id,human_readable_id,source,target,description,weight,combined_degree,text_unit_ids
0,2a5f9d86-1755-46cc-adc8-f7d78b99284e,0,绍兴,1129年,李清照在1129年追随高宗途中居住于绍兴,2.0,15,[ed0d7a7cec224017e5cfe7df292b6d6a031967471fba1...
1,6a68643a-acb6-4a9b-ac6f-d4a00a3d1c19,1,高宗,李清照,李清照，南宋著名女词人，词风婉约，被誉为“词中的女杰”。在1129年至1130年间，李清照经...,6.0,472,[ed0d7a7cec224017e5cfe7df292b6d6a031967471fba1...
2,15e689c4-27aa-453d-8243-dacd41d5d158,2,1129年,《清平乐》,李清照在1129年创作了反映个人心路历程的词《清平乐》,2.0,11,[ed0d7a7cec224017e5cfe7df292b6d6a031967471fba1...


In [34]:
print(relationship_df.head(3))

                                     id  human_readable_id source target  \
0  2a5f9d86-1755-46cc-adc8-f7d78b99284e                  0     绍兴  1129年   
1  6a68643a-acb6-4a9b-ac6f-d4a00a3d1c19                  1     高宗    李清照   
2  15e689c4-27aa-453d-8243-dacd41d5d158                  2  1129年  《清平乐》   

                                         description  weight  combined_degree  \
0                               李清照在1129年追随高宗途中居住于绍兴     2.0               15   
1  李清照，南宋著名女词人，词风婉约，被誉为“词中的女杰”。在1129年至1130年间，李清照经...     6.0              472   
2                        李清照在1129年创作了反映个人心路历程的词《清平乐》     2.0               11   

                                       text_unit_ids  
0  [ed0d7a7cec224017e5cfe7df292b6d6a031967471fba1...  
1  [ed0d7a7cec224017e5cfe7df292b6d6a031967471fba1...  
2  [ed0d7a7cec224017e5cfe7df292b6d6a031967471fba1...  


In [8]:
relationships[:3]

[Relationship(id='2a5f9d86-1755-46cc-adc8-f7d78b99284e', short_id='0', source='绍兴', target='1129年', weight=2.0, description='李清照在1129年追随高宗途中居住于绍兴', description_embedding=None, text_unit_ids=['ed0d7a7cec224017e5cfe7df292b6d6a031967471fba15766cc24f3e2a6ea1938a39a6e1ade1469d75968ff39c54f128bdb85fd50969224885963128a6a43933'], rank=15, attributes=None),
 Relationship(id='6a68643a-acb6-4a9b-ac6f-d4a00a3d1c19', short_id='1', source='高宗', target='李清照', weight=6.0, description='李清照，南宋著名女词人，词风婉约，被誉为“词中的女杰”。在1129年至1130年间，李清照经历了一段颠沛流离的生活。据史料记载，她跟随当时的南宋君主高宗，也因金兵入侵南迁，至两浙一带。在1130年2月，她仍在追随高宗在浙东地区辗转流亡。这一时期，她所见所感都深深影响着她的作品，使其词作充满了对失去的家园的深切怀念和对动荡岁月的感慨。这一段经历不仅展示了其坚韧不拔的个性，也为研究李清照的词作提供了重要的历史背景。', description_embedding=None, text_unit_ids=['ed0d7a7cec224017e5cfe7df292b6d6a031967471fba15766cc24f3e2a6ea1938a39a6e1ade1469d75968ff39c54f128bdb85fd50969224885963128a6a43933', 'ca7bfa460129b5b6c75a6ef3f29eada7ffe11870437687ff65e703ff00bf54307f66033d716c0dd728ddb9a1634ddfdfd8d2e4f66c62637e1841137af9b27158', 'a271f2f2

In [38]:
# 示例输入
query = "李清照你在哪里创作了《清平乐》"

# 调用函数
result = extract_entities_and_relationships_optimized(query, entity_df, relationship_df)

# 打印结果
if result:
    for item in result:
        print(f"实体: {item['entity']}")
        print(f"description: {item['entity_description']}")
        print("relationships:")
        for rel in item['relationships']:
            print(f"  - source: {rel['source']}, target: {rel['target']}, description: {rel['description']}")
        print("-" * 50)

实体: 《清平乐》
description: 李清照在1129年创作的词，内容反映她的个人心路历程和国难之忧
relationships:
  - source: 1129年, target: 《清平乐》, description: 李清照在1129年创作了反映个人心路历程的词《清平乐》
  - source: 《清平乐》, target: 年年雪里, description: 《清平乐》中的名句“年年雪里”回忆了过去的赏梅情景
  - source: 《清平乐》, target: 李清照, description: 李清照借《清平乐》词来表现自己的心态
--------------------------------------------------
实体: 清照
description: 李清照是宋代一位著名的女词人，她在中国文学史上占有重要地位。她的作品《一剪梅》等多篇诗词传诵至今，深受人们喜爱。李清照的诗词作品不仅展现了其深厚的文学造诣，还透露出宋代社会文化的特点，她的作品对后世文人有着深远的影响。
relationships:
  - source: 清照, target: 赵明诚, description: 李清照与赵明诚之间的夫妻关系不仅仅是一段普通的婚姻生活，而是一段充满了文学与艺术氛围的共同生活。他们都对文学艺术有着浓厚的兴趣，尤其是在诗词和书法方面，两人互相影响，共同研究，形成了深厚的感情和良好的合作关系。在中国文学史上，李清照不仅以其才女的身份著称，她的词作更是传唱千古，尤其是在宋代女性词人中占有举足轻重的地位。赵明诚作为她的丈夫，也是一位有学问的文人，他们的夫妻关系和共同生活对李清照的文学创作产生了积极的影响。通过他们的互动，可以窥见当时文人圈子中的关系与结构，以及宋代文化生活的一瞥。
  - source: 清照, target: 《一剪梅》, description: 李清照创作了《一剪梅》，反映其对赵明诚的思念
  - source: 益都, target: 清照, description: 李清照在益都居住并创作了多首诗词
  - source: 赵挺之, target: 清照, description: 赵挺之被追夺赠官，可能影响了李清照的家道中落
  - source: 清照, target: 晁补之, description:

In [39]:
result = extract_entities_and_relationships_optimized("你的丈夫是谁？", entity_df, relationship_df)
if result:
    for item in result:
        print(f"实体: {item['entity']}")
        print(f"description: {item['entity_description']}")
        print("relationships:")
        for rel in item['relationships']:
            print(f"  - source: {rel['source']}, target: {rel['target']}, description: {rel['description']}")
        print("-" * 50)

未在 query 中找到匹配的实体。


In [40]:
result = extract_entities_and_relationships_optimized("声声慢创作在什么时候？", entity_df, relationship_df)
if result:
    for item in result:
        print(f"实体: {item['entity']}")
        print(f"description: {item['entity_description']}")
        print("relationships:")
        for rel in item['relationships']:
            print(f"  - source: {rel['source']}, target: {rel['target']}, description: {rel['description']}")
        print("-" * 50)

实体: 声声慢
description: 近世的歌词形式之一，音律结构特点
relationships:
  - source: 李清照, target: 声声慢, description: 李清照在1147年创作了《声声慢》
--------------------------------------------------


In [41]:
result = extract_entities_and_relationships_optimized("武陵春的全文内容", entity_df, relationship_df)
if result:
    for item in result:
        print(f"实体: {item['entity']}")
        print(f"description: {item['entity_description']}")
        print("relationships:")
        for rel in item['relationships']:
            print(f"  - source: {rel['source']}, target: {rel['target']}, description: {rel['description']}")
        print("-" * 50)

实体: 武陵春
description: 李清照的一首词作品
relationships:
  - source: 易安居士, target: 武陵春, description: 易安居士即李清照创作了武陵春
  - source: 武陵春, target: 李清照, description: 李清照创作了词作《武陵春》
  - source: 李清照, target: 武陵春, description: 在金华期间，李清照创作了感情深沉的《武陵春》
--------------------------------------------------
实体: 春
description: 在李清照的词中，春天不仅是一个季节，还象征着国家和个人的命运
relationships:
  - source: 李清照, target: 春, description: 李清照用春天象征国家和个人的命运
--------------------------------------------------
实体: 武陵
description: 武陵是一个古地名，在李清照的诗歌中，这个地名被用来象征性地指代她的丈夫所在的地方，或者是她表达对丈夫思念之情的一个地点。李清照通过将“武陵”用作借指，创造了一种含蓄而深情的文学手法，以此抒发对心爱之人的缅怀与渴望。
relationships:
  - source: 武陵, target: 李清照, description: 李清照是中国宋代著名的女诗人，她在作品中提到的“武陵”不仅是一个具体的地点，更承载着深厚的文化象征意义。在她的心中，武陵代表着逐渐远去的心爱之人，这反映了她对丈夫深沉的思念与复杂的情感。通过对武陵的描绘，李清照巧妙地运用了地名作为情感的象征，表达了个人生活中的感慨及失落感。在这种文学创作手法中，我们可以看到李清照对爱情的忠诚与感情的细腻，以及她作为女性诗人在中国文学史上的重要地位。
  - source: 李清照, target: 武陵, description: 在《凤台上忆吹箫》中，武陵象征着作者丈夫所在的地方，表达了她的思念
李清照在《凤台上忆吹箫》中提及武陵，用以寓指丈夫所去的地方
  - source: 武陵, target: 秦楼, description: 武陵和秦楼在李清照的诗词中象徵着彼此思念的地

In [42]:
result = extract_entities_and_relationships_optimized("在金华创作了哪些作品？", entity_df, relationship_df)
if result:
    for item in result:
        print(f"实体: {item['entity']}")
        print(f"description: {item['entity_description']}")
        print("relationships:")
        for rel in item['relationships']:
            print(f"  - source: {rel['source']}, target: {rel['target']}, description: {rel['description']}")
        print("-" * 50)

实体: 金华
description: 金华是中国浙江省的一个地级市，在宋代女词人李清照的生平中占有特殊的地位。1134年，李清照因战乱避难至金华，直至1135年5月。在这段艰难的流亡生涯中，她的创作并未停歇，反而更加深刻地表达了她对国破家亡的愁苦。1135年，她在金华创作了《题八咏楼》，通过这首词作抒发了自己对过往生活的怀念和对现实悲剧的感慨。金华不仅是她晚年流落的地方，也是她诏令取《哲宗实录》的所在地，这反映了她即使在困顿时期，仍然保持着对文化和历史的热爱与追求。李清照在金华的生活虽然充满了苦难，但她在这里的文学创作，尤其是《题八咏楼》，至今仍被后人传颂，展现了她不屈不挠的精神和卓越的文学才华。
relationships:
  - source: 金华, target: 易安居士, description: 易安居士即李清照在金华陈氏第居住并创作
  - source: 金华, target: 1134年10月, description: 李清照于1134年10月抵达金华并居住在陈氏第
  - source: 金华, target: 1134年11月24日, description: 李清照在金华完成《打马图经》及其序言
  - source: 金华, target: 1134年11月, description: 李清照在金华作《打马赋》
  - source: 金华, target: 绍兴四年, description: 绍兴四年，李清照在金华作《打马图经序》
  - source: 易安居士, target: 金华, description: 易安居士，即李清照，是宋代著名的女词人。在她的一生中，由于历史的变迁和个人的遭遇，她离开了宋朝的都城临安，移居到金华。在金华，李清照不但居住，同时继续了她的文学创作，创作出了许多脍炙人口的词作。李清照以其细腻的情感和深邃的文学造诣，反映了宋代文人的生活状态与情感世界，也为后世留下了珍贵的文化财产。她所用的别号「易安居士」也因此与她的个人经历和文学成就一同被世人所铭记。
  - source: 金华, target: 李清照, description: 李清照在金华有词抒发孤独之感
  - source: 李清照, target: 金华, description: 李清照，中国宋代著名女词人，她的一生充满了传奇色彩。在

## 报告

In [9]:
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")

print(f"Report records: {len(report_df)}")
report_df.head(3)

Report records: 122


,id,human_readable_id,community,level,parent,children,title,summary,full_content,rank,rating_explanation,findings,full_content_json,period,size
0,455bee04dbf746b68cea169ac87f5836,112,112,2,69,[],李清照与赵明诚的金石学研究,本报告围绕宋代著名女诗人李清照及其丈夫赵明诚的金石学研究，特别是他们合著的《金石录》以及与文...,# 李清照与赵明诚的金石学研究\n\n本报告围绕宋代著名女诗人李清照及其丈夫赵明诚的金石学研...,7.0,赵明诚与李清照在金石学研究和文学方面的成就，对中国历史文化有深远影响。,[{'explanation': '《金石录》是由李清照和赵明诚共同整理，其记录了大量的金石...,"{\n ""title"": ""李清照与赵明诚的金石学研究"",\n ""summary...",2025-03-28,6
1,9f2fff76dba04d34abd944f0bfa743cd,113,113,2,69,[],归来堂与1115年的文化交流,归来堂是李清照与赵明诚学术交流与合作的关键地点，1115年象征着这对夫妇在艺术与学问上的共同...,# 归来堂与1115年的文化交流\n\n归来堂是李清照与赵明诚学术交流与合作的关键地点，11...,2.0,虽然这对夫妇对金石学的贡献被历史所记住，但其影响范围相对较小，主要局限于学术界。,[{'explanation': '1115年是李清照的个人艺术成就的标志性年份，她创作了著...,"{\n ""title"": ""归来堂与1115年的文化交流"",\n ""summar...",2025-03-28,2
2,72ee3526d41b4e3f8fbbe32485c29544,114,114,2,69,[],江宁与李清照的历史联系,江宁作为历史著名的古都，在文人李清照的生活与作品中占据了重要的地位。李清照在1128年2月到...,# 江宁与李清照的历史联系\n\n江宁作为历史著名的古都，在文人李清照的生活与作品中占据了重...,4.0,江宁与李清照之间的联系在中国历史和文化上具有一定的重要性。,[{'explanation': '江宁既是中国历史上的重要古都，也是许多文人墨客的创作寄托...,"{\n ""title"": ""江宁与李清照的历史联系"",\n ""summary"":...",2025-03-28,2


In [10]:
reports = read_indexer_reports(report_df, entity_embedding_df, COMMUNITY_LEVEL)
reports[:3]

[CommunityReport(id='455bee04dbf746b68cea169ac87f5836', short_id='112', title='李清照与赵明诚的金石学研究', community_id='112', summary='本报告围绕宋代著名女诗人李清照及其丈夫赵明诚的金石学研究，特别是他们合著的《金石录》以及与文人陆德夫的文学交流。此外，赵明诚在1129年前往行在的个人经历也被纳入考量。', full_content='# 李清照与赵明诚的金石学研究\n\n本报告围绕宋代著名女诗人李清照及其丈夫赵明诚的金石学研究，特别是他们合著的《金石录》以及与文人陆德夫的文学交流。此外，赵明诚在1129年前往行在的个人经历也被纳入考量。\n\n## 《金石录》的学术价值\n\n《金石录》是由李清照和赵明诚共同整理，其记录了大量的金石拓片及其解读，反映了二人对金石文物的深刻理解和广泛收集整理工作[数据：实体（391），关系（563）]。\n\n## 李清照的文学与金石学成就\n\n李清照不仅在诗词歌赋方面有卓越成就，同时也在文物研究与历史文化保存方面做出了贡献，这体现在她合著的《金石录》以及与陆德夫的文学交流中[数据：实体（391），关系（932）]。\n\n## 赵明诚与陆德夫的文学交流\n\n赵明诚与陆德夫之间的关系突显了宋代文学社群的交流模式。赵明诚不仅赞赏李清照的创作，还将她的词与自身的作品融合后呈现给陆德夫[数据：实体（860），关系（932）]。\n\n## 赵明诚在1129年的经历\n\n在1129年，赵明诚离开江宁，并在6月13日前往建康，这表明了他个人与行政事务的关联[数据：实体（919，311），关系（988，351，992）]。\n\n## 赵明诚与李清照的深刻连接\n\n无论是在专业的金石学合作还是日常生活的点滴中，赵明诚与李清照之间的关系深深植根于他们的作品和情感生活中[数据：实体（391，148），关系（563，168）]。', rank=7.0, full_content_embedding=None, attributes=None, size=None, period=None),
 CommunityReport(id='9f2fff76dba04d34abd944f0bfa743cd', short_id='113',

## 文本块

In [11]:
text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")

print(f"Text unit records: {len(text_unit_df)}")
text_unit_df.head(3)

Text unit records: 100


,id,human_readable_id,text,n_tokens,document_ids,entity_ids,relationship_ids,covariate_ids
0,ed0d7a7cec224017e5cfe7df292b6d6a031967471fba15...,1,绍兴（1129年，46岁，作品：1）\n1129年，46岁\n1129年 追随高宗于两浙，有...,1200,[03e4742412e1be85fbc7a7737fe0cc9da4241d2ce5e76...,"[7d8dfeeb-d5f6-4f25-ae86-f69c7994249e, 73125e5...","[2a5f9d86-1755-46cc-adc8-f7d78b99284e, 6a68643...",[]
1,ca7bfa460129b5b6c75a6ef3f29eada7ffe11870437687...,2,世的艰辛。当年那种赏梅的雅兴大减。这两句写的就是词人婚后的这段生活，表现的是一种百无聊赖、忧...,1027,[03e4742412e1be85fbc7a7737fe0cc9da4241d2ce5e76...,"[7d8dfeeb-d5f6-4f25-ae86-f69c7994249e, 73125e5...","[6a68643a-acb6-4a9b-ac6f-d4a00a3d1c19, feacc67...",[]
2,35db9c01730e44b834f55c18ffe1db83515f95c656b2d9...,3,7月 金人立刘豫为帝，清照作诗斥伪齐。作品：《咏史》\n\n绍兴（1131年3月 - 113...,127,[03e4742412e1be85fbc7a7737fe0cc9da4241d2ce5e76...,"[7d8dfeeb-d5f6-4f25-ae86-f69c7994249e, 73125e5...","[e6c0a66e-70f9-44da-bb35-ddc48882bcb8, 796f3ca...",[]


In [12]:
text_units = read_indexer_text_units(text_unit_df)
text_units[:3]

[TextUnit(id='ed0d7a7cec224017e5cfe7df292b6d6a031967471fba15766cc24f3e2a6ea1938a39a6e1ade1469d75968ff39c54f128bdb85fd50969224885963128a6a43933', short_id='0', text='绍兴（1129年，46岁，作品：1）\n1129年，46岁\n1129年\u3000追随高宗于两浙，有词写奔亡之苦。作品：《清平乐》（年年雪里）\n\n清平乐（1129年） 宋 · 李清照\n年年雪里，常插梅花醉，挼尽梅花无好意，赢得满衣清泪！\n今年海角天涯，萧萧两鬓生华。\n看取晚来风势，故应难看梅花。\n【评注】\n这是一首典型的赏梅词作，借不同时期的赏梅感爱写出了词人个人的心路历程：少年的欢乐，中年的幽怨，晚年的沧落，在词中都约略可见。词意含蓄蕴藉，感情悲切哀婉，以赏梅寄寓自己的今苦之感和永国之忧，感慨深沉。\n上阕回忆往昔赏梅生活，分为两层。“年年雪里，常插梅花醉 。”这两句抓住富有特征的生活细节生动地再现了词人早年赏梅的情景和兴致，表现出少女的纯真 、欢乐和闲适。她早年写下的咏梅词《渔家傲》中有句云 ：“雪里已知春信至，寒梅点缀琼枝腻⋯⋯共赏金尊沉绿蚁，莫辞醉，此花不与群花比 。”可作为“年年雪里，常插梅花醉”的注脚。接下来“挼尽梅花无好意 ，赢得满衣清泪”两句，心绪显然不同，虽然梅枝在手，却无好心情去赏玩，只是漫不经心地揉搓着。赏梅原本为的是排遣心头的忧伤，可是本来心情就不好，到头来不仅忧伤没有消除，反倒触景生情，激起更深的伤感，落得个“满衣清泪 ”。花还是昔日的花，然而花相似，人不同，物是人非，怎不使人伤心落泪呢？李清照婚后，夫妻志同道合、伉俩相得，生活美满幸福。但是，时常发生的短暂离别使她识尽离愁别苦。在婚后六、七年的时间里，李赵两家相继罹祸，紧接着就开始了长期的“屏居乡里”的生活。生活的坎坷使她屡处忧患，饱尝人世的艰辛。当年那种赏梅的雅兴大减。这两句写的就是词人婚后的这段生活，表现的是一种百无聊赖、忧伤怨恨的情绪。\n上阕四句回忆了作者两个生活阶段赏梅时的又一不同情景和心情。\n下阕以“今年”两字领起，同上阕的“年年”相对 。往年是“常插梅花醉”；即使是“挼尽梅花无好意”的时候，也多半为的是离别相思。眼前却截然不同了，“今

# 尝试graphRAG本身的search方法

## 自定义本地embedding模型

In [13]:
import requests
from typing import Any, List

class OpenaiApiType:
    """
    Enum-like class to define API types.
    """
    OpenAI = "openai"
    Azure = "azure"
    Custom = "custom"

class OpenAIEmbedding:
    def __init__(
        self,
        api_key: str,
        api_base: str,
        api_type: str,
        model: str,
        max_retries: int = 3,
    ):
        self.api_key = api_key
        self.api_base = api_base.rstrip("/")  # Ensure no trailing slash
        self.api_type = api_type
        self.model = model
        self.max_retries = max_retries

    def embed(self, text: str, **kwargs: Any) -> List[float]:
        url = f"{self.api_base}/embeddings"
        payload = {
            "model": self.model,
            "input": text,
            **kwargs,
        }
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}",
        }

        retries = 0
        while retries < self.max_retries:
            try:
                response = requests.post(url, json=payload, headers=headers)
                response.raise_for_status()  # Raise an exception for HTTP errors
                data = response.json()
                return data.get("data", [])[0].get("embedding", [])
            except requests.exceptions.RequestException as e:
                retries += 1
                if retries >= self.max_retries:
                    raise RuntimeError(f"Failed to generate embedding after {self.max_retries} retries: {e}")
                print(f"Retrying... ({retries}/{self.max_retries})")


In [14]:
import tiktoken
token_encoder = tiktoken.get_encoding("cl100k_base")

text_embedder = OpenAIEmbedding(
    api_key='ollama',
    api_base="http://localhost:11434/v1",
    api_type=OpenaiApiType.OpenAI,
    model='mxbai-embed-large',
    max_retries=20,
)

In [15]:
text = "李清照你的父亲是谁"
embedding = text_embedder.embed(text)
print(f"Embedding: {embedding}")

Embedding: [-0.04587041, 0.00746515, 0.05074231, 0.015815087, -0.03345231, -0.04500356, 0.017951166, 0.046151325, 0.016805328, 0.019730577, 0.043083917, 0.023942865, 0.019049339, -0.00855244, -0.073606096, -0.031414416, -0.023627525, -0.028932815, -0.05708194, 0.040650915, 0.042135827, 0.0041262223, -0.07222413, -0.019322377, 0.0037676403, 0.027726537, -0.021892449, -0.008191838, 0.06418192, 0.056086194, -0.06796903, 0.003948503, 0.014302877, -0.0058405935, -0.026621526, 0.010652543, 0.033079576, -0.016537178, -0.036137402, -0.053311903, 0.004390875, -0.04468908, 0.063471794, -0.052651856, -0.032443143, -0.010269888, -0.0066934177, -0.016524063, -0.008029126, -0.06616176, -0.0254616, 0.0028730126, 0.027936751, -0.010017751, 0.043522608, 0.008855017, 0.0013563463, 0.008562924, 0.012573169, 0.03483613, 0.007709181, 0.006691046, -0.012193698, -0.05135225, -0.0046411613, 0.051086143, -0.012788888, 0.0026270386, 0.017402047, -0.005393974, -0.008531303, 0.019698074, -0.014879573, -0.02341016

## 接入LanceDB

In [16]:
description_embedding_store = LanceDBVectorStore(
    collection_name="default-entity-description",
)
description_embedding_store.connect(db_uri=LANCEDB_URI)

context_builder负责执行一系列的  实体、关系、描述、源文本 等数据的提取任务

In [17]:
context_builder = LocalSearchMixedContext(
    entities=entities,
    entity_text_embeddings=description_embedding_store,
    text_embedder=text_embedder,
    text_units=text_units,
    community_reports=reports,
    relationships=relationships,
    covariates=None,
    token_encoder=token_encoder,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,
)

site-packages\graphrag\vector_stores\lancedb.py
- 在向量库中匹配时同步获取分数score
```py
def similarity_search_by_vector(
    self, query_embedding: list[float], k: int = 10, **kwargs: Any
) -> list[VectorStoreSearchResult]:
    """Perform a vector-based similarity search."""
    if self.query_filter:
        docs = (
            self.document_collection.search(
                query=query_embedding, vector_column_name="vector"
            )
            .where(self.query_filter, prefilter=True)
            .limit(k)
            .to_list()
        )
    else:
        docs = (
            self.document_collection.search(
                query=query_embedding, vector_column_name="vector"
            )
            .limit(k)
            .to_list()
        )
    return [
        VectorStoreSearchResult(
            document=VectorStoreDocument(
                id=doc["id"],
                text=doc["text"],
                vector=doc["vector"],
                attributes=json.loads(doc["attributes"]),
            ),
            score=1 - abs(float(doc["_distance"])),
        )
        for doc in docs
    ]
```



graphrag\query\context_builder\entity_extraction.py
对于抽取到实体基于分数（匹配距离）进行过滤
```py
def map_query_to_entities(
    query: str,
    text_embedding_vectorstore: BaseVectorStore,
    text_embedder: EmbeddingModel,
    all_entities_dict: dict[str, Entity],
    embedding_vectorstore_key: str = EntityVectorStoreKey.ID,
    include_entity_names: list[str] | None = None,
    exclude_entity_names: list[str] | None = None,
    k: int = 10,
    oversample_scaler: int = 2,
) -> list[Entity]:
    """Extract entities that match a given query using semantic similarity of text embeddings of query and entity descriptions."""
    if include_entity_names is None:
        include_entity_names = []
    if exclude_entity_names is None:
        exclude_entity_names = []
    all_entities = list(all_entities_dict.values())
    matched_entities = []
    if query != "":
        # get entities with highest semantic similarity to query
        # oversample to account for excluded entities
        search_results = text_embedding_vectorstore.similarity_search_by_text(
            text=query,
            text_embedder=lambda t: text_embedder.embed(t),
            k=k * oversample_scaler,
        )
        for result in search_results:
            if result.score < 0.6: # 在这里基于分数进行过滤
                continue
            if embedding_vectorstore_key == EntityVectorStoreKey.ID and isinstance(
                result.document.id, str
            ):
                matched = get_entity_by_id(all_entities_dict, result.document.id)
            else:
                matched = get_entity_by_key(
                    entities=all_entities,
                    key=embedding_vectorstore_key,
                    value=result.document.id,
                )
            if matched:
                matched_entities.append(matched)
    else:
        all_entities.sort(key=lambda x: x.rank if x.rank else 0, reverse=True)
        matched_entities = all_entities[:k]

    # filter out excluded entities
    if exclude_entity_names:
        matched_entities = [
            entity
            for entity in matched_entities
            if entity.title not in exclude_entity_names
        ]

    # add entities in the include_entity list
    included_entities = []
    for entity_name in include_entity_names:
        included_entities.extend(get_entity_by_name(all_entities, entity_name))
    return included_entities + matched_entities
```

## 测试效果

### 存在的实体匹配效果展示

In [18]:
# test1
text1 = "赵明诚"
context1 = context_builder.build_context(text1)

In [19]:
chunks1 = context1.context_chunks
records1 = context1.context_records

In [20]:
context1

ContextBuilderResult(context_chunks='-----Reports-----\nid|title|content\r\n11|李清照与宋代词坛|"# 李清照与宋代词坛\n\n社区围绕李清照，宋代著名女词人展开，其作品及别号易安影响深远。该社区涵盖了与李清照直接相关的个人，文学作品和评论家。这个社区不仅反映了宋代女性词风的特色，也揭示了后世学者对这一时期词作的评价和分析。\n\n## 李清照，易安的文化遗产\n\n李清照使用易安这一别号，并以其别具一格的婉约词风著称于世。她的个人生活和创作受到了广泛的关注和研究[数据：实体（77）；关系（97）]。她的作品，像《一剪梅》和《声声慢》，均展现了她婉约风格下的深情和对社会变迁的微妙反映[数据：实体（330，392）；关系（373，1082）]。\n\n## 陈廷焯对李清照作品的深刻评论\n\n陈廷焯作为明代文学评论家，对李清照词作的评价表现出了对她艺术成就和情感内涵的深入认识，至今仍被后世研究引为参考[数据：实体（738）；关系（816）]。\n\n## 易安与宋代女性词风关系\n\n易安的词作体现了宋代以来婉约派的典型风格，包括情感细腻和语言的精炼。她的作品代表了这一文学流派的高峰，对后世文学观念的塑造有着不可忽视的影响[数据：实体（772）；关系（832）]。\n\n## 对李清照词作评论的现代文化视角\n\n梁启超，徐釚等近代和现代评论家通过他们的作品对李清照的词进行了深入的分析，展现出了他们独到的鉴赏眼光和对情感表达的敏锐洞察力。他们的评论工作为理解宋代及李清照作品提供了宝贵的文化和学术视角[数据：实体（39，970）；关系（1080，1057）]。\n\n## 李清照作品中的情感与历史背景\n\n李清照通过她的词作反映了个人情感与当时社会的紧密联系。例如，《醉花阴》和《声声慢》中，她使用个人经历映射了社会动荡，展现了个人与历史的密切关系[数据：实体（760，392）]。\n\n## 李清照才华的文人间的识别和共鸣\n\n陆德夫等文人通过他们的交往和交流展现了对李清照词才的欣赏以及在文学创作上的互相影响。他们的行为反映了宋代文学社群的交流模式和对李清照才华的认可[数据：实体（860）；关系（932）]。"\r\n0|李清照与宋代文化社区|"# 李清照与宋代文化社区\n\n本报告聚焦于

In [21]:
chunks1,records1

('-----Reports-----\nid|title|content\r\n11|李清照与宋代词坛|"# 李清照与宋代词坛\n\n社区围绕李清照，宋代著名女词人展开，其作品及别号易安影响深远。该社区涵盖了与李清照直接相关的个人，文学作品和评论家。这个社区不仅反映了宋代女性词风的特色，也揭示了后世学者对这一时期词作的评价和分析。\n\n## 李清照，易安的文化遗产\n\n李清照使用易安这一别号，并以其别具一格的婉约词风著称于世。她的个人生活和创作受到了广泛的关注和研究[数据：实体（77）；关系（97）]。她的作品，像《一剪梅》和《声声慢》，均展现了她婉约风格下的深情和对社会变迁的微妙反映[数据：实体（330，392）；关系（373，1082）]。\n\n## 陈廷焯对李清照作品的深刻评论\n\n陈廷焯作为明代文学评论家，对李清照词作的评价表现出了对她艺术成就和情感内涵的深入认识，至今仍被后世研究引为参考[数据：实体（738）；关系（816）]。\n\n## 易安与宋代女性词风关系\n\n易安的词作体现了宋代以来婉约派的典型风格，包括情感细腻和语言的精炼。她的作品代表了这一文学流派的高峰，对后世文学观念的塑造有着不可忽视的影响[数据：实体（772）；关系（832）]。\n\n## 对李清照词作评论的现代文化视角\n\n梁启超，徐釚等近代和现代评论家通过他们的作品对李清照的词进行了深入的分析，展现出了他们独到的鉴赏眼光和对情感表达的敏锐洞察力。他们的评论工作为理解宋代及李清照作品提供了宝贵的文化和学术视角[数据：实体（39，970）；关系（1080，1057）]。\n\n## 李清照作品中的情感与历史背景\n\n李清照通过她的词作反映了个人情感与当时社会的紧密联系。例如，《醉花阴》和《声声慢》中，她使用个人经历映射了社会动荡，展现了个人与历史的密切关系[数据：实体（760，392）]。\n\n## 李清照才华的文人间的识别和共鸣\n\n陆德夫等文人通过他们的交往和交流展现了对李清照词才的欣赏以及在文学创作上的互相影响。他们的行为反映了宋代文学社群的交流模式和对李清照才华的认可[数据：实体（860）；关系（932）]。"\r\n0|李清照与宋代文化社区|"# 李清照与宋代文化社区\n\n本报告聚焦于宋代女词人李清照及其作品与生活环境的关系，探讨了她的词作如何反映了宋代

In [22]:
print(type(records1))
print("*"*100)
print(records1.keys())
print("*"*100)
print('reports:')
print(records1['reports'])
print("*"*100)
print('relationships:')
print(records1['relationships'])
print("*"*100)
print('entities:')
print(records1['entities'])
print("*"*100)
print('sources:')
print(records1['sources'])

<class 'dict'>
****************************************************************************************************
dict_keys(['reports', 'relationships', 'entities', 'sources'])
****************************************************************************************************
reports:
   id       title                                            content
0  11    李清照与宋代词坛  # 李清照与宋代词坛\n\n社区围绕李清照，宋代著名女词人展开，其作品及别号易安影响深远。该...
1   0  李清照与宋代文化社区  # 李清照与宋代文化社区\n\n本报告聚焦于宋代女词人李清照及其作品与生活环境的关系，探讨了...
****************************************************************************************************
relationships:
      id      source   target  \
0    406          明诚      李清照   
1    166         李清照       明诚   
2    384          明诚       易安   
3    933          易安       明诚   
4   1099          益都       明诚   
5    167          南渡       明诚   
6    938          明诚      醉花阴   
7    467          建康       明诚   
8    924         醉花阴       明诚   
9    350          明诚       江宁   
10   563          明诚    《

In [23]:
type(records1['reports']),type(records1['relationships']),type(records1['entities']),type(records1['sources'])
# type(records1['reports']),type(records1['entities']),type(records1['sources'])

(pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame)

In [24]:
# print(type(chunks1))
print(chunks1)

-----Reports-----
id|title|content
11|李清照与宋代词坛|"# 李清照与宋代词坛

社区围绕李清照，宋代著名女词人展开，其作品及别号易安影响深远。该社区涵盖了与李清照直接相关的个人，文学作品和评论家。这个社区不仅反映了宋代女性词风的特色，也揭示了后世学者对这一时期词作的评价和分析。

## 李清照，易安的文化遗产

李清照使用易安这一别号，并以其别具一格的婉约词风著称于世。她的个人生活和创作受到了广泛的关注和研究[数据：实体（77）；关系（97）]。她的作品，像《一剪梅》和《声声慢》，均展现了她婉约风格下的深情和对社会变迁的微妙反映[数据：实体（330，392）；关系（373，1082）]。

## 陈廷焯对李清照作品的深刻评论

陈廷焯作为明代文学评论家，对李清照词作的评价表现出了对她艺术成就和情感内涵的深入认识，至今仍被后世研究引为参考[数据：实体（738）；关系（816）]。

## 易安与宋代女性词风关系

易安的词作体现了宋代以来婉约派的典型风格，包括情感细腻和语言的精炼。她的作品代表了这一文学流派的高峰，对后世文学观念的塑造有着不可忽视的影响[数据：实体（772）；关系（832）]。

## 对李清照词作评论的现代文化视角

梁启超，徐釚等近代和现代评论家通过他们的作品对李清照的词进行了深入的分析，展现出了他们独到的鉴赏眼光和对情感表达的敏锐洞察力。他们的评论工作为理解宋代及李清照作品提供了宝贵的文化和学术视角[数据：实体（39，970）；关系（1080，1057）]。

## 李清照作品中的情感与历史背景

李清照通过她的词作反映了个人情感与当时社会的紧密联系。例如，《醉花阴》和《声声慢》中，她使用个人经历映射了社会动荡，展现了个人与历史的密切关系[数据：实体（760，392）]。

## 李清照才华的文人间的识别和共鸣

陆德夫等文人通过他们的交往和交流展现了对李清照词才的欣赏以及在文学创作上的互相影响。他们的行为反映了宋代文学社群的交流模式和对李清照才华的认可[数据：实体（860）；关系（932）]。"
0|李清照与宋代文化社区|"# 李清照与宋代文化社区

本报告聚焦于宋代女词人李清照及其作品与生活环境的关系，探讨了她的词作如何反映了宋代的文化氛围和个人情感。李清照的作品与多个实体有关，包括个人经历、创作背景和

### 不存在的实体匹配效果展示

In [25]:
# test2
text2 = "中南大学"
context2 = context_builder.build_context(text2)
chunks2 = context2.context_chunks
records2 = context2.context_records

In [26]:
records2

{'reports':    id          title                                            content
 0   6  李清照与南宋初期的流亡历程  # 李清照与南宋初期的流亡历程\n\n本报告概述了著名女词人李清照在金军侵扰下随宋高宗赵构辗...
 1  58     黄岩与海道的历史联系  # 黄岩与海道的历史联系\n\n社区以历史著名地名黄岩为中心，与海道有关联。黄岩在南宋时期是...,
 'relationships':     id source target     description  in_context
 0  606     黄岩     海道  黄岩可能是通往海道的地点之一        True,
 'entities':     id entity description  in_context
 0  573     海道       海上的路线        True,
 'sources':    id                                               text
 0  53  。遂解舟下，一日夜行三百里。比至，果大服茈胡、黄芩药，疟且痢，病危在膏肓。余悲泣，仓皇不忍问...}

In [27]:
print(records2['reports'])
print("*" * 100)
print('relationships:')
print(records2['relationships'])
print("*" * 100)
print('entities:')
print(records2['entities'])
print("*" * 100)
print('sources:')
print(records2['sources'])

   id          title                                            content
0   6  李清照与南宋初期的流亡历程  # 李清照与南宋初期的流亡历程\n\n本报告概述了著名女词人李清照在金军侵扰下随宋高宗赵构辗...
1  58     黄岩与海道的历史联系  # 黄岩与海道的历史联系\n\n社区以历史著名地名黄岩为中心，与海道有关联。黄岩在南宋时期是...
****************************************************************************************************
relationships:
    id source target     description  in_context
0  606     黄岩     海道  黄岩可能是通往海道的地点之一        True
****************************************************************************************************
entities:
    id entity description  in_context
0  573     海道       海上的路线        True
****************************************************************************************************
sources:
   id                                               text
0  53  。遂解舟下，一日夜行三百里。比至，果大服茈胡、黄芩药，疟且痢，病危在膏肓。余悲泣，仓皇不忍问...


中南大学是匹配不到的，可以基于此在部署时调换其它检索方法
下面的博家之祖的相关问题同样如此

In [28]:
# test3
text3 = "什么是博家之祖？"
context3 = context_builder.build_context(text3)
chunks3 = context3.context_chunks
records3 = context3.context_records

In [29]:
records3,chunks3

({'reports':   id       title                                            content
  0  0  李清照与宋代文化社区  # 李清照与宋代文化社区\n\n本报告聚焦于宋代女词人李清照及其作品与生活环境的关系，探讨了...,
  'relationships':      id source target                      description links  in_context
  0   786     柳永    李清照   李清照在词中引用柳永《凤栖梧》的句子来形容锦被胡乱摊开的样子     7        True
  1   519    李清照     归路              李清照在《如梦令》中提到不知归路的感受     7        True
  2   522    李清照     争渡                  李清照在《如梦令》中提到了争渡     7        True
  3   795    李清照     刘晨                   通过刘晨典故传达李清照的情感     7        True
  4   796    李清照     阮肇                   通过阮肇典故传达李清照的感情     7        True
  5   831    李清照  婉约派词作                  李清照的词作是婉约派的典型代表     7        True
  6   929    李清照    临高阁          《忆秦娥》词作中，李清照描述临高阁上观景的情景     7        True
  7   832    婉约派     易安        易安的词作体现了正宗的婉约派风格，清新浅易而不攲靡     1        True
  8   249    鹧鸪天     锁窗        《鹧鸪天》一诗中出现锁窗，反映了作者的孤独和闭塞感     1        True
  9   882     琥珀    浣溪沙              在《浣溪沙》中，琥珀用来形容词中的酒色     1        True
  10  

### 问题驱动的实体匹配

In [30]:
# test3
text3 = "武陵春写作于什么时候？在哪里写的"
context3 = context_builder.build_context(text3)
chunks3 = context3.context_chunks
records3 = context3.context_records
records3, chunks3

({'reports':   id       title                                            content
  0  0  李清照与宋代文化社区  # 李清照与宋代文化社区\n\n本报告聚焦于宋代女词人李清照及其作品与生活环境的关系，探讨了...,
  'relationships':      id source target                    description links  in_context
  0   701  《武陵春》     舴艋       《武陵春》中利用舴艋小船象征性地表达了作者的愁绪     2        True
  1   702  《武陵春》    舴艋舟        《武陵春》中利用舴艋舟象征性地表达了作者的愁绪     2        True
  2   709    李清照     轻舟         轻舟象征着李清照突如其来但并不强烈的出游愿望    11        True
  3   808    李清照     伤春              李清照在词中表达了对春天的无限伤感    11        True
  4   226     春茶    李清照        李清照在词中通过春茶表达了她对春天的喜爱和享受    11        True
  5   692    武陵春    李清照                  李清照创作了词作《武陵春》    11        True
  6   721    李清照    武陵春         在金华期间，李清照创作了感情深沉的《武陵春》    11        True
  7   142    李清照  又斜风细雨        李清照在《念奴娇 春情》中描绘了斜风细雨的景象    11        True
  8   145    李清照   征鸿过尽  李清照在《念奴娇 春情》中使用征鸿过尽来表达时光流逝和离愁    11        True
  9   149    李清照   新桐初引      李清照在《念奴娇 春情》中表达了春天引种新桐的盼望    11        True
  10  150    李清照   日高烟敛    李

### 匹配存在问题



In [31]:
# test3
text3 = "赵挺之跟你有什么关系？"
context3 = context_builder.build_context(text3)
chunks3 = context3.context_chunks
records3 = context3.context_records
records3, chunks3

({'reports':   id       title                                            content
  0  0  李清照与宋代文化社区  # 李清照与宋代文化社区\n\n本报告聚焦于宋代女词人李清照及其作品与生活环境的关系，探讨了...,
  'relationships':      id source target                      description links  in_context
  0   809    李清照      月              李清照在遥望淡云来往月疏疏中流露出愁绪     4        True
  1   339    李清照   黄昏疏雨         在黄昏疏雨中，李清照创作了表达春日细微情感的词作     4        True
  2   795    李清照     刘晨                   通过刘晨典故传达李清照的情感     4        True
  3   796    李清照     阮肇                   通过阮肇典故传达李清照的感情     4        True
  4   249    鹧鸪天     锁窗        《鹧鸪天》一诗中出现锁窗，反映了作者的孤独和闭塞感     1        True
  5   500   梅心惊破    赵明成                    词人因思念赵明成而感情波澜     1        True
  6   882     琥珀    浣溪沙              在《浣溪沙》中，琥珀用来形容词中的酒色     1        True
  7   814     庭院      月          庭院映照着月光，提供了李清照内敛情感的表达方式     1        True
  8   254     秋风     玉井       玉井在诗中可能与秋风联合使用，共同营造一种凄凉的意境     1        True
  9   245     芭蕉   伤心之情  在《添字丑奴儿》中，芭蕉树与阴雨天气共同构成了李清照的伤心之情     1        True
  10  

In [32]:
# test3
text3 = "赵挺之"
context3 = context_builder.build_context(text3)
chunks3 = context3.context_chunks
records3 = context3.context_records
records3, chunks3

({'reports':   id       title                                            content
  0  0  李清照与宋代文化社区  # 李清照与宋代文化社区\n\n本报告聚焦于宋代女词人李清照及其作品与生活环境的关系，探讨了...,
  'relationships':      id source target                      description links  in_context
  0   786     柳永    李清照   李清照在词中引用柳永《凤栖梧》的句子来形容锦被胡乱摊开的样子     8        True
  1   809    李清照      月              李清照在遥望淡云来往月疏疏中流露出愁绪     8        True
  2   145    李清照   征鸿过尽    李清照在《念奴娇 春情》中使用征鸿过尽来表达时光流逝和离愁     8        True
  3   519    李清照     归路              李清照在《如梦令》中提到不知归路的感受     8        True
  4   522    李清照     争渡                  李清照在《如梦令》中提到了争渡     8        True
  5   795    李清照     刘晨                   通过刘晨典故传达李清照的情感     8        True
  6   796    李清照     阮肇                   通过阮肇典故传达李清照的感情     8        True
  7   831    李清照  婉约派词作                  李清照的词作是婉约派的典型代表     8        True
  8   249    鹧鸪天     锁窗        《鹧鸪天》一诗中出现锁窗，反映了作者的孤独和闭塞感     1        True
  9   882     琥珀    浣溪沙              在《浣溪沙》中，琥珀用来形容词中的酒色     1        True
  10  

赵挺之在可视化知识图谱中可以搜到

# 不使用lancedb，自定义查询方法

参考graphrag-visualizer项目中的内容，仅考虑 实体本身 & 实体关系

In [43]:
INPUT_DIR = "../graphRag/output"
ENTITY_TABLE = "entities"
RELATIONSHIP_TABLE = "relationships"

## 实体
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
print(f"Entity count: {len(entity_df)}")
## 关系
relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
print(f"Relationship count: {len(relationship_df)}")

entity_df.head(3),relationship_df.head(3)

Entity count: 1021
Relationship count: 1138


(                                     id  human_readable_id  title    type  \
 0  7d8dfeeb-d5f6-4f25-ae86-f69c7994249e                  0     绍兴     GEO   
 1  73125e53-678d-4ca9-b526-7679bf230aec                  1     高宗  PERSON   
 2  615e99dc-26e7-4a59-b424-b628ea596305                  2  1129年    DATE   
 
                                          description  \
 0  绍兴是中国浙江省的一个历史名城，宋代时期就已经非常重要。李清照，作为宋代著名的女词人，她的生...   
 1  宋高宗赵构，南宋初期的皇帝，同时也是宋真宗的第九子。在金军入侵北宋后期和南宋初期，国家动荡不...   
 2  1129年，即建炎三年，李清照46岁。这一年对她来说意义重大，不仅因为她在这一年创作了晚期的...   
 
                                        text_unit_ids  frequency  degree    x  \
 0  [ed0d7a7cec224017e5cfe7df292b6d6a031967471fba1...          4       7  0.0   
 1  [ed0d7a7cec224017e5cfe7df292b6d6a031967471fba1...          9       9  0.0   
 2  [ed0d7a7cec224017e5cfe7df292b6d6a031967471fba1...          5       8  0.0   
 
      y  
 0  0.0  
 1  0.0  
 2  0.0  ,
                                      id  human_readable_id source target  \
 0  2a5f9d86

In [44]:
from collections import defaultdict

# 假设 entity_df 和 relationship_df 已经加载
# entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
# relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")

def extract_entities_and_relationships_optimized(query, entity_df, relationship_df):
    # Step 1: 创建实体标题的集合，用于快速查找
    entity_titles = set(entity_df['title'])

    # 提取 query 中匹配的实体
    entities_in_query = [entity for entity in entity_titles if entity in query]

    if not entities_in_query:
        print("未在 query 中找到匹配的实体。")
        return None

    # Step 2: 获取匹配实体的 description
    matched_entities = entity_df[entity_df['title'].isin(entities_in_query)][['title', 'description']]
    matched_entities_dict = dict(zip(matched_entities['title'], matched_entities['description']))

    # Step 3: 创建关系的快速查找表
    relationships_grouped = defaultdict(list)
    for _, row in relationship_df.iterrows():
        relationships_grouped[row['source']].append({
            "source": row['source'],
            "target": row['target'],
            "description": row['description']
        })
        relationships_grouped[row['target']].append({
            "source": row['source'],
            "target": row['target'],
            "description": row['description']
        })

    # Step 4: 构建结果
    result = []
    for entity in entities_in_query:
        entity_description = matched_entities_dict.get(entity, "")
        related_relationships = relationships_grouped.get(entity, [])

        result.append({
            "entity": entity,
            "entity_description": entity_description,
            "relationships": related_relationships
        })

    return result


In [45]:
# 示例输入
query = "李清照你在哪里创作了《清平乐》"

# 调用函数
result = extract_entities_and_relationships_optimized(query, entity_df, relationship_df)

# 打印结果
if result:
    for item in result:
        print(f"实体: {item['entity']}")
        print(f"description: {item['entity_description']}")
        print("relationships:")
        for rel in item['relationships']:
            print(f"  - source: {rel['source']}, target: {rel['target']}, description: {rel['description']}")
        print("-" * 50)

实体: 《清平乐》
description: 李清照在1129年创作的词，内容反映她的个人心路历程和国难之忧
relationships:
  - source: 1129年, target: 《清平乐》, description: 李清照在1129年创作了反映个人心路历程的词《清平乐》
  - source: 《清平乐》, target: 年年雪里, description: 《清平乐》中的名句“年年雪里”回忆了过去的赏梅情景
  - source: 《清平乐》, target: 李清照, description: 李清照借《清平乐》词来表现自己的心态
--------------------------------------------------
实体: 清照
description: 李清照是宋代一位著名的女词人，她在中国文学史上占有重要地位。她的作品《一剪梅》等多篇诗词传诵至今，深受人们喜爱。李清照的诗词作品不仅展现了其深厚的文学造诣，还透露出宋代社会文化的特点，她的作品对后世文人有着深远的影响。
relationships:
  - source: 清照, target: 赵明诚, description: 李清照与赵明诚之间的夫妻关系不仅仅是一段普通的婚姻生活，而是一段充满了文学与艺术氛围的共同生活。他们都对文学艺术有着浓厚的兴趣，尤其是在诗词和书法方面，两人互相影响，共同研究，形成了深厚的感情和良好的合作关系。在中国文学史上，李清照不仅以其才女的身份著称，她的词作更是传唱千古，尤其是在宋代女性词人中占有举足轻重的地位。赵明诚作为她的丈夫，也是一位有学问的文人，他们的夫妻关系和共同生活对李清照的文学创作产生了积极的影响。通过他们的互动，可以窥见当时文人圈子中的关系与结构，以及宋代文化生活的一瞥。
  - source: 清照, target: 《一剪梅》, description: 李清照创作了《一剪梅》，反映其对赵明诚的思念
  - source: 益都, target: 清照, description: 李清照在益都居住并创作了多首诗词
  - source: 赵挺之, target: 清照, description: 赵挺之被追夺赠官，可能影响了李清照的家道中落
  - source: 清照, target: 晁补之, description:

In [46]:
result = extract_entities_and_relationships_optimized("你的丈夫是谁？", entity_df, relationship_df)
if result:
    for item in result:
        print(f"实体: {item['entity']}")
        print(f"description: {item['entity_description']}")
        print("relationships:")
        for rel in item['relationships']:
            print(f"  - source: {rel['source']}, target: {rel['target']}, description: {rel['description']}")
        print("-" * 50)

未在 query 中找到匹配的实体。


In [47]:
result = extract_entities_and_relationships_optimized("声声慢创作在什么时候？", entity_df, relationship_df)
if result:
    for item in result:
        print(f"实体: {item['entity']}")
        print(f"description: {item['entity_description']}")
        print("relationships:")
        for rel in item['relationships']:
            print(f"  - source: {rel['source']}, target: {rel['target']}, description: {rel['description']}")
        print("-" * 50)

实体: 声声慢
description: 近世的歌词形式之一，音律结构特点
relationships:
  - source: 李清照, target: 声声慢, description: 李清照在1147年创作了《声声慢》
--------------------------------------------------


In [48]:
result = extract_entities_and_relationships_optimized("武陵春的全文内容", entity_df, relationship_df)
if result:
    for item in result:
        print(f"实体: {item['entity']}")
        print(f"description: {item['entity_description']}")
        print("relationships:")
        for rel in item['relationships']:
            print(f"  - source: {rel['source']}, target: {rel['target']}, description: {rel['description']}")
        print("-" * 50)

实体: 武陵春
description: 李清照的一首词作品
relationships:
  - source: 易安居士, target: 武陵春, description: 易安居士即李清照创作了武陵春
  - source: 武陵春, target: 李清照, description: 李清照创作了词作《武陵春》
  - source: 李清照, target: 武陵春, description: 在金华期间，李清照创作了感情深沉的《武陵春》
--------------------------------------------------
实体: 春
description: 在李清照的词中，春天不仅是一个季节，还象征着国家和个人的命运
relationships:
  - source: 李清照, target: 春, description: 李清照用春天象征国家和个人的命运
--------------------------------------------------
实体: 武陵
description: 武陵是一个古地名，在李清照的诗歌中，这个地名被用来象征性地指代她的丈夫所在的地方，或者是她表达对丈夫思念之情的一个地点。李清照通过将“武陵”用作借指，创造了一种含蓄而深情的文学手法，以此抒发对心爱之人的缅怀与渴望。
relationships:
  - source: 武陵, target: 李清照, description: 李清照是中国宋代著名的女诗人，她在作品中提到的“武陵”不仅是一个具体的地点，更承载着深厚的文化象征意义。在她的心中，武陵代表着逐渐远去的心爱之人，这反映了她对丈夫深沉的思念与复杂的情感。通过对武陵的描绘，李清照巧妙地运用了地名作为情感的象征，表达了个人生活中的感慨及失落感。在这种文学创作手法中，我们可以看到李清照对爱情的忠诚与感情的细腻，以及她作为女性诗人在中国文学史上的重要地位。
  - source: 李清照, target: 武陵, description: 在《凤台上忆吹箫》中，武陵象征着作者丈夫所在的地方，表达了她的思念
李清照在《凤台上忆吹箫》中提及武陵，用以寓指丈夫所去的地方
  - source: 武陵, target: 秦楼, description: 武陵和秦楼在李清照的诗词中象徵着彼此思念的地

In [49]:
result = extract_entities_and_relationships_optimized("在金华创作了哪些作品？", entity_df, relationship_df)
if result:
    for item in result:
        print(f"实体: {item['entity']}")
        print(f"description: {item['entity_description']}")
        print("relationships:")
        for rel in item['relationships']:
            print(f"  - source: {rel['source']}, target: {rel['target']}, description: {rel['description']}")
        print("-" * 50)

实体: 金华
description: 金华是中国浙江省的一个地级市，在宋代女词人李清照的生平中占有特殊的地位。1134年，李清照因战乱避难至金华，直至1135年5月。在这段艰难的流亡生涯中，她的创作并未停歇，反而更加深刻地表达了她对国破家亡的愁苦。1135年，她在金华创作了《题八咏楼》，通过这首词作抒发了自己对过往生活的怀念和对现实悲剧的感慨。金华不仅是她晚年流落的地方，也是她诏令取《哲宗实录》的所在地，这反映了她即使在困顿时期，仍然保持着对文化和历史的热爱与追求。李清照在金华的生活虽然充满了苦难，但她在这里的文学创作，尤其是《题八咏楼》，至今仍被后人传颂，展现了她不屈不挠的精神和卓越的文学才华。
relationships:
  - source: 金华, target: 易安居士, description: 易安居士即李清照在金华陈氏第居住并创作
  - source: 金华, target: 1134年10月, description: 李清照于1134年10月抵达金华并居住在陈氏第
  - source: 金华, target: 1134年11月24日, description: 李清照在金华完成《打马图经》及其序言
  - source: 金华, target: 1134年11月, description: 李清照在金华作《打马赋》
  - source: 金华, target: 绍兴四年, description: 绍兴四年，李清照在金华作《打马图经序》
  - source: 易安居士, target: 金华, description: 易安居士，即李清照，是宋代著名的女词人。在她的一生中，由于历史的变迁和个人的遭遇，她离开了宋朝的都城临安，移居到金华。在金华，李清照不但居住，同时继续了她的文学创作，创作出了许多脍炙人口的词作。李清照以其细腻的情感和深邃的文学造诣，反映了宋代文人的生活状态与情感世界，也为后世留下了珍贵的文化财产。她所用的别号「易安居士」也因此与她的个人经历和文学成就一同被世人所铭记。
  - source: 金华, target: 李清照, description: 李清照在金华有词抒发孤独之感
  - source: 李清照, target: 金华, description: 李清照，中国宋代著名女词人，她的一生充满了传奇色彩。在

-  问题发现：

    - 实体匹配还是存在问题
        - 书名符号 《声声慢》匹配不到  只能匹配 声声慢
        - 冗余匹配
    - 实体关系的文本内容存在重复的情况，可能影响模型的表达
    - 实体关系溢出（如"李清照"本身）
-  解决思路：

    -  重构实体匹配逻辑，引入分词匹配
    -  后续重建知识图谱，在提示词中增加限制